In [ ]:
import itertools
import networkx as nx
from datetime import datetime

import numpy as np
import random
from scipy.stats import truncnorm
import math

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

import pandas as pd
import random

C:\Users\camil\AppData\Local\Temp\ipykernel_14040\2540983689.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [ ]:
state = 0

#read csv
households = pd.read_csv('households.csv')
pro_contacts_adults = pd.read_csv('pro_contacts_adults.csv')
pro_contacts_children = pd.read_csv('pro_contacts_children.csv')

In [ ]:
#get all the different job_cat from pro_contacts_adults
job_cat = pro_contacts_adults['job_cat']
#remove duplicates
job_cat = list(dict.fromkeys(job_cat))
print(job_cat)


[nan, 'Indus_other', 'Administration_schools', 'Indus_food', 'Transportation', 'Agriculture_fishing', 'Services_other', 'Shops_other', 'Construction', 'Health', 'Hotel_Restaurant', 'Shops_market_food']


In [ ]:
#create empty graph
G = nx.Graph()


In [ ]:
#create a graph using preferential attachment
def preferential_attachment_graph(degrees, names):
    #transform names and degrees into a list
    names = list(names)
    degrees = list(degrees)


    # Create an empty graph
    graph = nx.Graph()
    nodes = []
    # Add nodes with names and degrees
    for i in range(len(names)):
        graph.add_node(names[i])
        nodes.append([names[i], degrees[i]])

    # Create edges using preferential attachment
    tot_degree = (2 * sum(n[1] for n in nodes))
    num_nodes = len(nodes)

    while len(nodes) > 1:
        
        # Select two random nodes
        node1,degree1 = random.choice(nodes)
        
        node2,degree2 = random.choice(nodes)
        

        # Skip if the nodes are the same or already connected
        if node1 == node2 or graph.has_edge(node1, node2):
            continue
        else :

        # Calculate the product of their degrees
            degree_product = degree1 * degree2

            # Attach the nodes with probability proportional to their degree products
            attachment_probability = degree_product / tot_degree
            
            if random.random() < attachment_probability:
                
                graph.add_edge(node1, node2)

            # Remove the nodes if they have no more edges to attach
            print("degree1",degree1,'/',graph.degree(node1))
            if graph.degree(node1) == degree1:
                
                nodes.remove([node1,degree1])
                
            if graph.degree(node2) == degree2:
                
                nodes.remove([node2,degree2])
                
               

    return graph

In [ ]:
#get the households and ids of adults and children put them in a new dataframe
households_adults = pro_contacts_adults[['household_id', 'adult_id']]
households_children = pro_contacts_children[['household_id', 'child_id']]
households_adults = households_adults.rename(columns={'adult_id': 'person_id'})
households_children = households_children.rename(columns={'child_id': 'person_id'})

#add households_children to the bottom of households_adults
households2 = pd.concat([households_adults, households_children], ignore_index=True)

#add an edge between each person in the same household
households2=households2.groupby('household_id')
groups =  households2.groups
for group in groups:
    
    for pair in itertools.combinations(groups[group], 2):
        G.add_edge(pair[0], pair[1])



In [ ]:
#adults contacts
adults_contacts = pro_contacts_adults[['adult_id', 'company_id', 'pro_contacts']]
adults_contacts = adults_contacts.groupby('company_id')
adults_groups = adults_contacts.groups
for group in adults_groups:
    nb_contacts = {}
    for adult in adults_groups[group]:
    
        nb_contacts[adult] = adults_contacts.get_group(group).loc[adult]['pro_contacts']
        if nb_contacts[adult]>=len(adults_groups[group]):
            #add an edge between adult and all the other adults in the same company
            for other_adult in adults_groups[group]:
                if other_adult != adult:
                    G.add_edge(adult, other_adult)
            #add nb_contact_left random edges between adult and other adults from other companies
            nb_contact_left = nb_contacts[adult] - len(adults_groups[group])
            if nb_contact_left > 0:
                for i in range(int(nb_contact_left)):
                    other_adult = random.choice(list(G.nodes()))
                    while other_adult in adults_groups[group]:
                        other_adult = random.choice(list(G.nodes()))
                    G.add_edge(adult, other_adult)
        else:
            #add nb_contacts random edges between adult and other adults in the same company
            for i in range(int(nb_contacts[adult])):
                other_adult = random.choice(adults_groups[group])
                while other_adult == adult:
                    other_adult = random.choice(adults_groups[group])
                G.add_edge(adult, other_adult)
        
            
  
        

#preferential attachment


In [ ]:
#children contacts
children_contacts = pro_contacts_children[['child_id', 'school_id', 'school_contacts']]
children_contacts = children_contacts.groupby('school_id')
children_groups = children_contacts.groups
for group in children_groups:
    for child in children_groups[group]:
        nb_contacts = children_contacts.get_group(group).loc[child]['school_contacts']
        if nb_contacts>=len(children_groups[group]):
            #add an edge between child and all the other children in the same school
            for other_child in children_groups[group]:
                if other_child != child:
                    G.add_edge(child, other_child)
            #add nb_contact_left random edges between child and other children from other schools
            nb_contact_left = nb_contacts - len(children_groups[group])
            if nb_contact_left > 0:
                for i in range(int(nb_contact_left)):
                    other_child = random.choice(list(G.nodes()))
                    while other_child in children_groups[group]:
                        other_child = random.choice(list(G.nodes()))
                    G.add_edge(child, other_child)
        else:
            #add nb_contacts random edges between child and other children in the same school
            for i in range(int(nb_contacts)):
                other_child = random.choice(children_groups[group])
                while other_child == child:
                    other_child = random.choice(children_groups[group])
                G.add_edge(child, other_child)
            

In [ ]:
#save the graph
nx.write_gexf(G, "graph.gexf")


In [ ]:
from utils import *

#get random nodes from the graph
nodes = random.sample(list(G.nodes()), 10)
print(nodes)
for node in nodes:
    print(get_job(node))
    job = get_job(node)
    print(get_job_cat(job))
    print(get_household(node))

[1516, 7606, 1307, 7116, 6140, 222, 7893, 8831, 1712, 8130]
Construction
construction
614
Student
Student or unemployed
471
nan
Student or unemployed
532
Student
Student or unemployed
97
Shops_other
market_services
2982
Shops_other
market_services
91
Student
Student or unemployed
670
Student
Student or unemployed
1387
Administration_schools
non_market_services
695
Student
Student or unemployed
834
